# Westeros Tutorial Part 2a.i - Introducing emissions

In the first part, the model chose not base the power system on wind power as electricity from  wind turbines was more expensive than electricity produced from coal. However, now we introduce emissions to investigate the impact of climate policy. Let's see what happens then.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform(name = 'local')

In [ ]:
model = 'Westeros Electrified'

base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'cooling tech','llustrating cooling tech using addon',
                  keep_solution=False)
scen.check_out()

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Retrieve parameters to perform subsequent addition of parameters

In [ ]:
#year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'
gdp_profile = pd.Series([1., 1.5, 1.9], index=pd.Index([700, 710, 720], name='Time'))

## Defining input for different parameters 

We add `water supply` as new level of water supply and `freshwater_supply` & `saline_supply` as commodities

In [ ]:
scen.add_set("commodity", ["freshwater_supply", "saline_supply"])
scen.add_set("level", ["water_supply"])

In [ ]:
base_input = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_origin': country,
    'commodity': 'electricity',
    'time': 'year',
    'time_origin': 'year',
}

base_output = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_dest': country,
    'time': 'year',
    'time_dest': 'year', 
    'unit': '%',
}


base_capacity_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'time': 'year',
    'unit': '%',
}

base_technical_lifetime = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'y',
}

base_inv_cost = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'USD/GWa',
}

base_fix_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'unit': 'USD/GWa',
}

base_var_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'unit': 'USD/GWa',
}

### addon technology - interlinking technologies
In order to demonstrate the use of this feature, we will be adding a new demand for water to this scenario. This demand will be met by the coal_ppl via three cooling technologies, the addon technology.

We will therefore go through the following steps:

### Define a new demand for heat.
Add new technologies:
1. Once-through cooling by freshwater - `coal_ppl_ot_fresh`
2. Closed loop cooling by freshwater - `coal_ppl_cl_fresh`
3. Once through cooling seawater `coal_ppl_ot_saline`
4. Air cooling `coal_ppl_air`

Link these cooling tehcnologies to the `coal_ppl` using the addon feature.

#### Step1: Define a new demand


# Define a new commodity `heat`
scen.add_set("commodity", ["water"])

# Add heat demand at the useful level
water_demand = pd.DataFrame({
        'node': country,
        'commodity': 'water',
        'level': 'useful',
        'year': [700, 710, 720],
        'time': 'year',
        'value': (500 * gdp_profile).round(),
        'unit': '-',
    })
scen.add_par("demand", water_demand)

#### Step2: Define new technologies inlcuding a passout-turbine along with a grid and an in-house heat-distribution system

In [ ]:
tec = 'extract_freshwater_supply'

scen.add_set('technology', tec)

ext_fresh_water = make_df(base_output, technology=tec, commodity='freshwater', 
                   level='water_supply', value=1.0)

scen.add_par('output', ext_fresh_water)

tec = 'extract__freshwater_instream'

ext_freswater_instream  = make_df(base_output, technology=tec, commodity='freshwater', 
                   level='water_supply', value=1.0)

scen.add_par('output', ext_freswater_instream)


tec = 'cooling_tech 

cool_tech = make_df(base_output, technology=tec, commodity='freshwater', 
                   level='water_supply', value=1.0)

Add an addon technology here 

In [ ]:
scen.add_set('addon', 'cool_tech')

In [ ]:
type_addon = 'ot_fresh'
addon = 'cool_tech'
tec = 'coal_ppl'
scen.add_cat('addon', type_addon, addon)
scen.add_set('map_tec_addon', pd.DataFrame({'technology': tec,
                                            'type_addon': [type_addon]}))

type_addon = 'cl_fresh'
addon = 'cool_tech'
tec = 'coal_ppl'
scen.add_cat('addon', type_addon, addon)
scen.add_set('map_tec_addon', pd.DataFrame({'technology': tec,
                                            'type_addon': [type_addon]}))

type_addon = 'ot_saline'
addon = 'cool_tech'
tec = 'coal_ppl'
scen.add_cat('addon', type_addon, addon)
scen.add_set('map_tec_addon', pd.DataFrame({'technology': tec,
                                            'type_addon': [type_addon]}))

type_addon = 'air'
addon = 'cool_tech'
tec = 'coal_ppl'
scen.add_cat('addon', type_addon, addon)
scen.add_set('map_tec_addon', pd.DataFrame({'technology': tec,
                                            'type_addon': [type_addon]}))


In [ ]:

# Index for `addon_conversion` is ['node', 'technology', 'year_vtg',
#                                  'year_act', 'mode', 'time',
#                                  'type_addon', 'value', 'unit']
df = pd.DataFrame({'node': country,
                   'technology': tec,
                   'year_vtg': vintage_years,
                   'year_act': act_years,
                   'mode': 'standard',
                   'time': 'year',
                   'type_addon': type_addon,
                   'value': 5,
                   'unit': '%'})
scen.add_par('addon_conversion', df)